In [1]:
#Dependencies
import pandas as pd
# SQL Alchemy
from sqlalchemy import create_engine

In [2]:
# Create Engine for Pitchfork
pitchfork_database_path = "./Resources/pitchfork.sqlite"
engine = create_engine(f"sqlite:///{pitchfork_database_path}")
conn = engine.connect()

In [3]:
engine.table_names()



['artists', 'content', 'genres', 'labels', 'reviews', 'years']

In [4]:
pitchfork_data = pd.read_sql("SELECT * FROM reviews", conn)
pitchfork_data.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017


In [5]:
#Store Spotify CSV into Dataframe
spotify_csv = "Resources/spotify_top10s.csv"
spotify_data_df = pd.read_csv(spotify_csv, encoding = "ISO-8859-1")
spotify_data_df.head()

#Create new data with select columns:
new_spotify_data_df = spotify_data_df[['title','artist','year']]
new_spotify_data_df

,title,artist,year
0,"Hey, Soul Sister",Train,2010
1,Love The Way You Lie,Eminem,2010
2,TiK ToK,Kesha,2010
3,Bad Romance,Lady Gaga,2010
4,Just the Way You Are,Bruno Mars,2010
...,...,...,...
598,Find U Again (feat. Camila Cabello),Mark Ronson,2019
599,Cross Me (feat. Chance the Rapper & PnB Rock),Ed Sheeran,2019
600,"No Brainer (feat. Justin Bieber, Chance the Ra...",DJ Khaled,2019
601,Nothing Breaks Like a Heart (feat. Miley Cyrus),Mark Ronson,2019


In [9]:
#spotify_data_df.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [6]:
#Use pandas to load csv converted DataFrame into database
new_spotify_data_df.to_sql(name='top10', con=engine,if_exists='append',index=False)

In [7]:
#Confirm data has been added by querying the table
pd.read_sql_query('select * from top10', con=engine).head()

,title,artist,year
0,"Hey, Soul Sister",Train,2010
1,Love The Way You Lie,Eminem,2010
2,TiK ToK,Kesha,2010
3,Bad Romance,Lady Gaga,2010
4,Just the Way You Are,Bruno Mars,2010


In [20]:
pitchfork_data = pd.read_sql("SELECT * FROM years", conn)

#columns in table
print(list(pitchfork_data.columns))

['reviewid', 'year']


In [38]:
#Investigating merge between both dataframes
initial_reviews = pd.read_sql("SELECT * FROM reviews", conn)
initial_reviews
reviews = initial_reviews[['reviewid','artist','score','author']]
reviews

,reviewid,artist,score,author
0,22703,massive attack,9.3,nate patrin
1,22721,krallice,7.9,zoe camp
2,22659,uranium club,7.3,david glickman
3,22661,"kleenex, liliput",9.0,jenn pelly
4,22725,taso,8.1,kevin lozano
...,...,...,...,...
18388,1535,coldcut,8.9,james p. wisdom
18389,1341,cassius,4.8,james p. wisdom
18390,5376,mojave 3,6.3,jason josephes
18391,2413,don caballero,7.2,james p. wisdom


In [43]:
content = pd.read_sql("SELECT * FROM content", conn)
content

,reviewid,content
0,22703,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,"Eight years, five albums, and two EPs in, the ..."
2,22659,Minneapolis’ Uranium Club seem to revel in bei...
3,22661,Kleenex began with a crash. It transpired one ...
4,22725,It is impossible to consider a given release b...
...,...,...
18388,1535,The marketing guys of yer average modern megac...
18389,1341,"Well, it's been two weeks now, and I guess it'..."
18390,5376,"Out of Tune is a Steve Martin album. Yes, I'l..."
18391,2413,"Well, kids, I just went back and re-read my re..."


In [48]:
content['reviewid'].nunique()

18389

In [50]:
#merge dataframes on reviewid column
content_reviews = reviews.merge(content, how='inner',on='reviewid')
content_reviews

,reviewid,artist,score,author,content
0,22703,massive attack,9.3,nate patrin,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,krallice,7.9,zoe camp,"Eight years, five albums, and two EPs in, the ..."
2,22659,uranium club,7.3,david glickman,Minneapolis’ Uranium Club seem to revel in bei...
3,22661,"kleenex, liliput",9.0,jenn pelly,Kleenex began with a crash. It transpired one ...
4,22725,taso,8.1,kevin lozano,It is impossible to consider a given release b...
...,...,...,...,...,...
18396,1535,coldcut,8.9,james p. wisdom,The marketing guys of yer average modern megac...
18397,1341,cassius,4.8,james p. wisdom,"Well, it's been two weeks now, and I guess it'..."
18398,5376,mojave 3,6.3,jason josephes,"Out of Tune is a Steve Martin album. Yes, I'l..."
18399,2413,don caballero,7.2,james p. wisdom,"Well, kids, I just went back and re-read my re..."


In [58]:
#Checking for duplicates
duplicate_reviews = content_reviews[content_reviews.duplicated()]
duplicate_reviews

,reviewid,artist,score,author,content
12117,9417,easy star all-stars,7.0,joe tangari,\r\n A song-for-song reggae cover of Radioh...
12118,9417,easy star all-stars,7.0,joe tangari,\r\n A song-for-song reggae cover of Radioh...
12119,9417,easy star all-stars,7.0,joe tangari,\r\n A song-for-song reggae cover of Radioh...
12121,9505,various artists,8.2,tim finney,\nOn the one hand it is a largely superfluous ...
12122,9505,various artists,8.2,tim finney,\nOn the one hand it is a largely superfluous ...
12123,9505,various artists,8.2,tim finney,\nOn the one hand it is a largely superfluous ...
12125,9499,the blood brothers,6.2,jason crock,"When we last left our heroes, the Blood Brothe..."
12126,9499,the blood brothers,6.2,jason crock,"When we last left our heroes, the Blood Brothe..."
12127,9499,the blood brothers,6.2,jason crock,"When we last left our heroes, the Blood Brothe..."
12129,9460,xasthur,7.8,brandon stosuy,Strange things are a foot in the bowels of hel...


In [63]:
#Drop columns with identical values in all columns
content_reviews.drop_duplicates(subset = ["reviewid", "artist", "score", "author", "content"])

,reviewid,artist,score,author,content
0,22703,massive attack,9.3,nate patrin,"“Trip-hop” eventually became a ’90s punchline,..."
1,22721,krallice,7.9,zoe camp,"Eight years, five albums, and two EPs in, the ..."
2,22659,uranium club,7.3,david glickman,Minneapolis’ Uranium Club seem to revel in bei...
3,22661,"kleenex, liliput",9.0,jenn pelly,Kleenex began with a crash. It transpired one ...
4,22725,taso,8.1,kevin lozano,It is impossible to consider a given release b...
...,...,...,...,...,...
18396,1535,coldcut,8.9,james p. wisdom,The marketing guys of yer average modern megac...
18397,1341,cassius,4.8,james p. wisdom,"Well, it's been two weeks now, and I guess it'..."
18398,5376,mojave 3,6.3,jason josephes,"Out of Tune is a Steve Martin album. Yes, I'l..."
18399,2413,don caballero,7.2,james p. wisdom,"Well, kids, I just went back and re-read my re..."
